In [1]:
import numpy as np
import pandas as pd 

import gc
import time
import joblib
import cudf 

In [2]:
permutationimportance = pd.read_csv("../input/permutationimportance-amex/permutationimportance_repeats3_meantransposed_v1.csv")

In [3]:
#permutationimportance_neg = permutationimportance[permutationimportance["importance"] < 0].column.values

## Constants:

In [4]:
RANDOM_STATE = 42

NAN_VALUE = -127

#k-folds:
FOLDS = 5

## Processing and feature engineering:

In [6]:
def process_and_feature(df):
    df = df.sort_values(['customer_ID','S_2']) 
    df = df.reset_index(drop=True)
    
    df["S_2"] = cudf.to_datetime(df["S_2"])
    
    # setting up permutation importance variables:
    permutationimportance_neg = permutationimportance[:50].column.values
    top3 = permutationimportance[-3:].column.values
    top5 = permutationimportance[-5:].column.values
    top7 = permutationimportance[-7:].column.values
    top10 = permutationimportance[-10:].column.values
    top15 = permutationimportance[-15:].column.values
    top25 = permutationimportance[-25:].column.values

    # set np seed:
    np.random.seed(RANDOM_STATE)
    
    # sorting on customer ID and then time, to use last properly
    df = df.sort_values(['customer_ID','S_2'])
    
    
    cat_col = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
    num_col = [col for col in df.columns if col not in (cat_col + ["customer_ID", "S_2","target"])]
    all_col = num_col + cat_col
    
    def convert64to32(df):
    #float64 and int64 to 32 : https://www.kaggle.com/code/ragnar123/amex-lgbm-dart-cv-0-7977
        float64_cols = list(df.dtypes[df.dtypes == "float64"].index)
        for col in float64_cols:
            df[col] = df[col].astype("float32")
        int64_cols = list(df.dtypes[df.dtypes == "int64"].index)
        for col in int64_cols:
            df[col] = df[col].astype("int32")
        return df

    # mean and sum of the different sort of variables
    D_col_num = []
    S_col_num = []
    P_col_num = []
    B_col_num = []
    R_col_num = []
    for col in df.columns:
        if col not in (cat_col + ["customer_ID", "S_2"]):
            if "D_" in col:
                D_col_num.append(col)
            elif "S_" in col:
                S_col_num.append(col)
            elif "P_" in col:
                P_col_num.append(col)
            elif "B_" in col:
                B_col_num.append(col)
            elif "R_" in col:
                R_col_num.append(col)
        else:
            pass
    df["D_mean"] = df[D_col_num].mean(axis=1)
    df["D_sum"] = df[D_col_num].sum(axis=1)
    df["S_mean"] = df[S_col_num].mean(axis=1)
    df["S_sum"] = df[S_col_num].sum(axis=1)
    df["P_mean"] = df[P_col_num].mean(axis=1)
    df["P_sum"] = df[P_col_num].sum(axis=1)
    df["B_mean"] = df[B_col_num].mean(axis=1)
    df["B_sum"] = df[B_col_num].sum(axis=1)
    df["R_mean"] = df[R_col_num].mean(axis=1)
    df["R_sum"] = df[R_col_num].sum(axis=1)

    var_groups_cols = ["D_mean","S_mean","P_mean","B_mean","R_mean",
                       "D_sum","S_sum","P_sum","B_sum","R_sum",
                      ]
    df_var_groups = df.groupby("customer_ID")[var_groups_cols].agg(["last","max","mean"])
    df_var_groups.columns = ['_'.join(x) for x in df_var_groups.columns]
    df_var_groups = convert64to32(df_var_groups)

    ######################
    # P_2 ratios:   
    #P_2÷B_9
    df["P2divB9"] = df["P_2"]/df["B_9"]
    
#     P2_ratio_col = ["D_42", "B_3","B_1","B_4","D_39"]
#     P2_ratio_col_finished = []
#     for col in P2_ratio_col:
#         df[f"P2-{col}-ratio"] = df[col]/df["P_2"]
#         P2_ratio_col_finished.append(f"P2-{col}-ratio")
    P2_ratio_col_finished = ["P2divB9"]
    df_P2_ratios = df.groupby("customer_ID")[P2_ratio_col_finished].agg(["last","min","max","mean"])
    df_P2_ratios.columns = ['_'.join(x) for x in df_P2_ratios.columns]
    #################################

    
    # count:
    df_count = df.groupby("customer_ID")["S_2"].agg(["count"])
    for col in df_count.columns:
        df_count[col] = df_count[col].astype("int8")
    
    # count non NaN values:
#     df_nonNANcount = df.groupby("customer_ID")[all_col][df[all_col] != -127].count().sort_index()
#     df_nonNANcount.columns = [f"{col}_nonNAN" for col in df_nonNANcount.columns]
#     for col in df_nonNANcount.columns:
#         df_nonNANcount[col] = df_nonNANcount[col].astype("int8")
        
    
    # time difference:
    df_date = df.groupby("customer_ID")[["S_2"]].agg(["first","last"])
    df_date.columns = ['_'.join(x) for x in df_date.columns]
    df_date["S_2_diff"] = ((df_date["S_2_last"] - df_date["S_2_first"])/np.timedelta64(1, 'D'))
    df_date["S_2_diff"] = df_date["S_2_diff"].astype("int16")
    df_date = df_date.drop(columns=["S_2_first","S_2_last"])
    
    # time variables
    df["weekday"] = df.S_2.dt.weekday
    df["month"] = df.S_2.dt.month
    df["percofyear"] = np.round(((df.S_2.dt.dayofyear / 365) * 100), 1)
    df["percofmonth"] = np.round(((df.S_2.dt.day / df.S_2.dt.days_in_month) * 100), 1)
    df["quarter"] = df.S_2.dt.quarter
    time_col = ["weekday","month","percofyear","percofmonth","quarter"]
    df_time = df.groupby("customer_ID")[time_col].agg(["last"])
    df_time.columns = ['_'.join(x) for x in df_time.columns]
    df_time = convert64to32(df_time)
    
    # 2nd last variables
    all_2nd_col = all_col + var_groups_cols + P2_ratio_col_finished
    df_2ndlast = df.groupby("customer_ID")[all_2nd_col].nth(-2) 
    df_2ndlast.columns = [str(x) + "_2ndlast" for x in df_2ndlast.columns]
    # https://www.kaggle.com/competitions/amex-default-prediction/discussion/332880#1830894 @Carl McBride Ellis
    df_2ndlast = convert64to32(df_2ndlast)
    
    # categorical columns:
    df_cat = df.groupby("customer_ID")[cat_col].agg(["first","last","nunique"]) # ['count', 'last', 'nunique']
    df_cat.columns = ['_'.join(x) for x in df_cat.columns]
    df_cat = convert64to32(df_cat)
    
    # Numerical columns:
    df_num = df.groupby("customer_ID")[num_col].agg(["first","last","min","max","mean"]) #'mean', 'std', 'min', 'max', 'last'
    df_num.columns = ['_'.join(x) for x in df_num.columns]
    df_num = convert64to32(df_num)
    
    ######
    global last_col
    last_col = all_col #+ var_groups_cols
    last_col = [f"{col}_last" for col in last_col]
    last_col = [col for col in last_col if col not in permutationimportance_neg]
    
    
    ############# Concat ######################################################
    print("concat")
    df = cudf.concat([df_date,df_time, df_count,df_cat, df_num, df_var_groups, df_2ndlast,df_P2_ratios], axis=1) 
    
    
    del df_date,df_time, df_count,df_cat, df_num, df_var_groups, df_2ndlast, df_P2_ratios
    gc.collect()
    
    # fill in NaN for 2ndlast with only 1 customer:
    
    for col in all_2nd_col:
        df[f"{col}_2ndlast"].fillna(df[f"{col}_last"], inplace=True) 
    
    # second stage:
    
    for col in num_col:
        df[f"{col}_last-mean"] = df[f"{col}_last"] - df[f"{col}_mean"]
#         df[f"{col}_MaxdivMin"] = df[f"{col}_max"] - df[f"{col}_min"]
        # new ones V22
        df[f"{col}_last-first"] = df[f"{col}_last"] - df[f"{col}_first"]
        df[f"{col}_lastDIVfirst"] = (df[f"{col}_last"]+0.0001) / (df[f"{col}_first"]+0.0001) # +0.0001 to prevent inf

        
#     for col in all_2nd_col:
#         df[f"{col}_last-2ndlast"] = df[f"{col}_last"] - df[f"{col}_2ndlast"]
#         df = df.drop(columns=f"{col}_2ndlast")

    df = df.reset_index()
    
    # permutation importance features:
    
    top_dic = {"top3":top3, "top5":top5,"top7":top7,"top10":top10,"top15":top15,"top25":top25}
    for top in top_dic:
        df[f"{top}_mean"] = df[top_dic[top]].mean(axis=1)
        df[f"{top}_sum"] = df[top_dic[top]].sum(axis=1)
        df[f"{top}_min"] = df[top_dic[top]].min(axis=1)
        df[f"{top}_max"] = df[top_dic[top]].max(axis=1)
    ####    
    for i in top5:
        for x in top5:
            if i == x:
                pass
            else:
                df[f"{i}DIV{x}"] = (df[i]+0.0001)/(df[x]+0.0001)
                df[f"{i}MIN{x}"] = df[i]-df[x]
                
        df[f"{i}_top5perc"] = df[i]/ df["top5_sum"] * 100
    float64_cols = list(df.dtypes[df.dtypes == "float64"].index)
    for col in float64_cols:
        df[col] = df[col].astype("float32") 
        
    # dropping columns:
    df = df.drop(columns=permutationimportance_neg)
    
    return df



In [8]:
def train_process():
    # read in data:
    train = cudf.read_parquet("../input/amex-data-integer-dtypes-parquet-format/train.parquet")
    # Categorical columns are label encoded and missing values are replaced by -1 @Raddar
    
    train["customer_ID"] = train["customer_ID"].str[-16:].str.hex_to_int().astype("int64") # not needed for test

    # feature engineering:
    train = process_and_feature(train) 
    
    #del train
    gc.collect()
    train_labels = cudf.read_csv("../input/amex-default-prediction/train_labels.csv")
    train_labels["customer_ID"] = train_labels["customer_ID"].str[-16:].str.hex_to_int().astype("int64") # not needed for test
    
    # add target labels:
    train_labels.target = train_labels.target.astype('int8')
    train = train.merge(train_labels, on="customer_ID")
    train = train.sort_values("customer_ID").reset_index(drop=True)

    del train_labels
    gc.collect()
    print(f"shape:{train.shape}")
    train = train.to_pandas() # free up GPU
    
    
    gc.collect()
    return train

In [9]:
%%time
train_final = train_process() #1710

concat


/opt/conda/lib/python3.7/site-packages/cupy/_statistics/order.py:111: RuntimeWarning: All-NaN slice encountered
  warnings.warn('All-NaN slice encountered', RuntimeWarning)
/opt/conda/lib/python3.7/site-packages/cupy/_statistics/order.py:142: RuntimeWarning: All-NaN slice encountered
  warnings.warn('All-NaN slice encountered', RuntimeWarning)


shape:(458913, 1710)


In [16]:
train_final.to_parquet("train_processed.parquet")

In [17]:
del train_final
gc.collect()

21

# test:

In [18]:
# clear GPU memory
# torch.cuda.empty_cache()
# cuda.select_device(0)
# cuda.close()
# cuda.select_device(0)
gc.collect()

21

In [19]:
%%time 
chunks = 8
test_rows = 11363762
groupby_test_rows = 924621
start_chunk = 0
length_chunk = int(groupby_test_rows/chunks)

test_full_id = cudf.DataFrame()

for chunk in range(chunks):

    print(f"chunk:{chunk}")
    test = cudf.read_parquet("../input/amex-data-integer-dtypes-parquet-format/test.parquet")
    test = test.sort_values(["customer_ID","S_2"])
    test = test.reset_index(drop=True)
    if (chunk+1) == chunks:
        end_chunk = test_rows +1
    else:
        end_chunk = test.loc[test["customer_ID"] == test.groupby("customer_ID").agg(["last"]).iloc[(length_chunk*(chunk+1))].index[0]].tail(1).index[0]
        end_chunk += 1
    print(f"chunk#{chunk}, start chunk id {start_chunk}, end_chunk id {end_chunk}")
    test = test.iloc[start_chunk:end_chunk]
    gc.collect()
    print("loaded chunk")
    test = process_and_feature(test)
    gc.collect()
    print("feature engineered")
    test = test.sort_values("customer_ID").reset_index(drop=True)
    test_full_id = test_full_id.append(test[["customer_ID"]])
    test.to_parquet(f"test_processed_chunk{chunk}.parquet")
    
    del test
    gc.collect()
    start_chunk = end_chunk

chunk:0
chunk#0, start chunk id 0, end_chunk id 1420901
loaded chunk
concat


/opt/conda/lib/python3.7/site-packages/cupy/_statistics/order.py:111: RuntimeWarning: All-NaN slice encountered
  warnings.warn('All-NaN slice encountered', RuntimeWarning)
/opt/conda/lib/python3.7/site-packages/cupy/_statistics/order.py:142: RuntimeWarning: All-NaN slice encountered
  warnings.warn('All-NaN slice encountered', RuntimeWarning)


feature engineered
chunk:1
chunk#1, start chunk id 1420901, end_chunk id 2841209
loaded chunk
concat
feature engineered
chunk:2
chunk#2, start chunk id 2841209, end_chunk id 4261572
loaded chunk
concat
feature engineered
chunk:3
chunk#3, start chunk id 4261572, end_chunk id 5681053
loaded chunk
concat
feature engineered
chunk:4
chunk#4, start chunk id 5681053, end_chunk id 7102344
loaded chunk
concat
feature engineered
chunk:5
chunk#5, start chunk id 7102344, end_chunk id 8523150
loaded chunk
concat
feature engineered
chunk:6
chunk#6, start chunk id 8523150, end_chunk id 9944645
loaded chunk
concat
feature engineered
chunk:7
chunk#7, start chunk id 9944645, end_chunk id 11363763
loaded chunk
concat
feature engineered
